In [2]:
from ecephys_analysis_modules.common.static_gratings_sdv import StaticGratings
import pandas as pd
import h5py
import numpy as np

In [3]:
nwb_path = '/allen/aibs/mat/Kael/ecephys_data/mouse412792.spikes.nwb'
sg = StaticGratings(nwb_path=nwb_path)

Computing responses


/local1/workspace/ecephys_analysis_modules/ecephys_analysis_modules/common/static_gratings_sdv.py:124: RuntimeWarning: Mean of empty slice.
  running_speed.running_speed[ind] = self.dxcm[np.where(self.dxcm_ts>row_stim.start)[0][0]:np.where(self.dxcm_ts>row_stim.end)[0][0]].mean()
/local1/.local/miniconda3/envs/neuropixels_3.6/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


Getting null distribution


/local1/.local/miniconda3/envs/neuropixels_3.6/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/local1/.local/miniconda3/envs/neuropixels_3.6/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [15]:
def compare_peak_data(actual_df, expected_h5):
    failed = []
    peak_grp = expected_h5['peak']
    sid_lu = {sid: i for i, sid in enumerate(peak_grp['cell_specimen_id'])}
    
    actual_df.set_index('cell_specimen_id', inplace=True)
    actual_ids = set(actual_df.index.values.astype(np.uint))
    expected_ids = set(peak_grp['cell_specimen_id'][()])
    if actual_ids != expected_ids:
        print('specimen_ids do not match.')
        return False
    for sid in actual_ids:
        actual_pd = actual_df.loc[str(sid)]
        expected_indx = sid_lu[sid]
        for col in ['pref_ori_sg', 'pref_sf_sg', 'pref_phase_sg', 'num_pref_trials_sg', 'responsive_sg', 
                    'g_osi_sg', 'sfdi_sg', 'reliability_sg', 'lifetime_sparseness_sg', 'fit_sf_sg', 
                    'fit_sf_ind_sg', 'sf_low_cutoff_sg', 'sf_high_cutoff_sg', 'run_pval_sg', 'run_mod_sg', 
                    'run_resp_sg', 'stat_resp_sg', 'lifetime_sparseness_dg']:
        
            if np.isnan(actual_pd[col]):
                if not np.isnan(peak_grp[col][expected_indx]):
                    failed.append('{}, {}> {} != {}'.format(sid, col, actual_pd[col], peak_grp[col][expected_indx]))
            else:
                if actual_pd[col] != peak_grp[col][expected_indx]:
                    failed.append('{}, {}> {} != {}'.format(sid, col, actual_pd[col], peak_grp[col][expected_indx]))
    if failed:
        print(failed)
        return False
    
    return True


assert(compare_peak_data(sg.peak.copy(), h5py.File('expected/mouse412792.static_grating.h5', 'r')))

In [16]:
def compare_mean_sweeps(actual_df, expected_h5):
    failed = []
    mse_grp = expected_h5['mean_sweep_events']
    actual_ids = actual_df.columns.values.astype(np.uint64)
    expected_ids = mse_grp['specimen_ids'][()]
    if set(actual_ids) != set(expected_ids):
        print('specimen_ids do not match.')
        return False
    
    sid_lu = {sid: i for i, sid in enumerate(mse_grp['specimen_ids'])}
    for sid in actual_ids:
        expected_indx = sid_lu[sid]
        expected_vals = mse_grp['data'][:,expected_indx]
        actual_vals = actual_df[str(sid)].values
        if not np.all(expected_vals == actual_vals):
            failed.append(sid)
    
    if failed:
        print('failed specimen ids: {}'.format(failed))
        return False
    return True       

        
assert(compare_mean_sweeps(sg.mean_sweep_events.copy(), 
                           h5py.File('expected/mouse412792.static_grating.h5', 'r')))

In [43]:
def compare_p_sweeps(actual_df, expected_h5):
    failed = []
    spv_grp = expected_h5['sweep_p_values']
    actual_ids = actual_df.columns.values.astype(np.uint64)
    expected_ids = spv_grp['specimen_ids'][()]
    if set(actual_ids) != set(expected_ids):
        print('specimen_ids do not match.')
        return False
    
    sid_lu = {sid: i for i, sid in enumerate(spv_grp['specimen_ids'])}
    for sid in actual_ids:
        expected_indx = sid_lu[sid]
        expected_vals = spv_grp['data'][:,expected_indx]
        actual_vals = actual_df[str(sid)].values
        # print(actual_vals)
        # if not np.all(expected_vals == actual_vals):
        if not np.allclose(expected_vals, actual_vals, atol=1.0e-3):     
            print(sid)
            #print(expected_vals)
            #print(actual_vals)
            for i in range(6000):
                # print(np.abs(expected_vals[i] - actual_vals[i]))
                if np.abs(expected_vals[i] - actual_vals[i]) > 1.0e-3:
                    # print("HERE")
                    print(expected_vals[i], actual_vals[i])
                    #return False
                    
            
            #return False
            failed.append(sid)
    
    if failed:
        print('failed specimen ids: {}'.format(failed))
        return False
    return True       

 
#sg.sweep_p_values
compare_p_sweeps(sg.sweep_p_values.copy(), 
                        h5py.File('expected/mouse412792.static_grating.h5', 'r'))
#assert(compare_p_sweeps(sg.sweep_p_values.copy(), 
#                        h5py.File('expected/mouse412792.static_grating.h5', 'r')))

165
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.0988 0.0972
0.

0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.0536 0.0578
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.0536 0.0578
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.0536 0.0578
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.0536 0.0578
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.0536 0.0578
0.2308

0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.0107 0.0118
0.0107 0.0118
0.0536 0.0578
0.0536 0.0578
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.0536 0.0578
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.0107 0.0118
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.0536 0.0578
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.2308 0.2373
0.2308 0.2373
0.0107 0.0118
0.2308 0.2373
0.0536 0.0578
0.0536

0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11
0.108 0.11

0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301 0.0264
0.0301

...

0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042

0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209 0.0197
0.3042 0.2918
0.3042 0.2918
0.3042 0.2918
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.0209 0.0197
0.3042 0.2918
0.0209

0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202 0.1153
0.1202

0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
0.0268 0.0308
266
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.0082 0.0098
0.

False

In [91]:
def compare_sweep_events(actual_df, expected_h5):
    print(actual_df)
    
assert(compare_sweep_events(sg.sweep_events.copy(), 
                           h5py.File('expected/mouse412792.static_grating.h5', 'r')))

failed specimen ids: [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 254, 256, 259, 260, 262, 264, 266]


/local1/.local/miniconda3/envs/neuropixels_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  from ipykernel import kernelapp as app
/local1/.local/miniconda3/envs/neuropixels_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


AssertionError: 